In [41]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostRegressionModel, XGBoostRegressor}
import org.apache.spark.ml.evaluation.{RegressionEvaluator}
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [3]:
val schema =
    StructType(Array(
      StructField("vendor_id", FloatType),
      StructField("passenger_count", FloatType),
      StructField("trip_distance", FloatType),
      StructField("pickup_longitude", FloatType),
      StructField("pickup_latitude", FloatType),
      StructField("rate_code", FloatType),
      StructField("store_and_fwd", FloatType),
      StructField("dropoff_longitude", FloatType),
      StructField("dropoff_latitude", FloatType),
      StructField("fare_amount", FloatType),
      StructField("trip_time",LongType),
      StructField("year", IntegerType),
      StructField("month", IntegerType),
      StructField("day", FloatType),
      StructField("day_of_week", FloatType),
      StructField("is_weekend", FloatType),
      StructField("hour", FloatType),
      StructField("h_distance",DoubleType)
    ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema: org.apache.spark.sql.types.StructType = StructType(StructField(vendor_id,FloatType,true), StructField(passenger_count,FloatType,true), StructField(trip_distance,FloatType,true), StructField(pickup_longitude,FloatType,true), StructField(pickup_latitude,FloatType,true), StructField(rate_code,FloatType,true), StructField(store_and_fwd,FloatType,true), StructField(dropoff_longitude,FloatType,true), StructField(dropoff_latitude,FloatType,true), StructField(fare_amount,FloatType,true), StructField(trip_time,LongType,true), StructField(year,IntegerType,true), StructField(month,IntegerType,true), StructField(day,FloatType,true), StructField(day_of_week,FloatType,true), StructField(is_weekend,FloatType,true), StructField(hour,FloatType,true), StructField(h_distance,DoubleType,true))


In [15]:
import ml.dmlc.xgboost4j.scala.spark.rapids.CrossValidator


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ml.dmlc.xgboost4j.scala.spark.rapids.CrossValidator


In [4]:
var df = spark.read.option("inferSchema", "false").option("header", true).schema(schema).csv("s3://bucketcjm/data/taxismall.csv") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [vendor_id: float, passenger_count: float ... 16 more fields]


In [5]:
df.show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+-------------+----------------+---------------+---------+-------------+-----------------+----------------+-----------+---------+----+-----+----+-----------+----------+----+------------------+
|vendor_id|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code|store_and_fwd|dropoff_longitude|dropoff_latitude|fare_amount|trip_time|year|month| day|day_of_week|is_weekend|hour|        h_distance|
+---------+---------------+-------------+----------------+---------------+---------+-------------+-----------------+----------------+-----------+---------+----+-----+----+-----------+----------+----+------------------+
|      1.0|            1.0|          1.1|         -74.005|         40.743|      1.0|          0.0|          -73.992|          40.754|        6.5|      341|2016|    5|15.0|        1.0|       1.0|20.0|1.5971032399653422|
|      1.0|            1.0|          1.1|         -74.003|         40.718|      1.0|          0.0|          -74.004|        

In [6]:
def dropUseless(dataFrame: DataFrame): DataFrame = {
    dataFrame.drop(
      "year",
      "passenger_count",
      "month",
      "vendor_id",
      "is_weekend",
      "trip_distance",
      "store_and_fwd")
  }


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dropUseless: (dataFrame: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [7]:
var tdf = dropUseless(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tdf: org.apache.spark.sql.DataFrame = [pickup_longitude: float, pickup_latitude: float ... 9 more fields]


In [8]:
tdf.cache
tdf.createOrReplaceTempView("taxi")
spark.catalog.cacheTable("taxi")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [pickup_longitude: float, pickup_latitude: float ... 9 more fields]


In [9]:
tdf.select("h_distance","fare_amount","trip_time").describe().show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+-----------------+
|summary|        h_distance|       fare_amount|        trip_time|
+-------+------------------+------------------+-----------------+
|  count|             29413|             29413|            29413|
|   mean| 4.326592293617547|15.115734539162325|1005.094108047462|
| stddev|4.1461764497000955|10.568815892745402|657.1706875514186|
|    min|1.0007950502042966|               2.5|              133|
|    max| 40.37790919118657|              97.0|            14476|
+-------+------------------+------------------+-----------------+



In [36]:
%%sql
select hour, avg(fare_amount)
from taxi
group by hour order by hour 

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [37]:
tdf.groupBy("hour").avg("fare_amount", "h_distance").orderBy("hour").show(24)

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [34]:
%%sql 
select trip_time from taxi

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
var featureNames = Array("h_distance", "pickup_longitude","pickup_latitude","dropoff_longitude", "dropoff_latitude","rate_code","hour", "day_of_week","trip_time")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

featureNames: Array[String] = Array(h_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, rate_code, hour, day_of_week, trip_time)


In [13]:
val Array(traindf, evaldf) = tdf.randomSplit(Array(0.7, 0.3), 5043)
traindf.show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

traindf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [pickup_longitude: float, pickup_latitude: float ... 9 more fields]
evaldf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [pickup_longitude: float, pickup_latitude: float ... 9 more fields]
+----------------+---------------+---------+-----------------+----------------+-----------+---------+----+-----------+----+------------------+
|pickup_longitude|pickup_latitude|rate_code|dropoff_longitude|dropoff_latitude|fare_amount|trip_time| day|day_of_week|hour|        h_distance|
+----------------+---------------+---------+-----------------+----------------+-----------+---------+----+-----------+----+------------------+
|      -74.034996|      40.615997|      1.0|           -73.97|          40.751|       41.5|     2186|25.0|        5.0|20.0|15.989421723650755|
|         -74.029|      40.622997|      1.0|        -73.91901|          40.769|       48.0|     2191|20.0|        7.0|20.0| 18.74339347011618|
|      -74.020996|

In [17]:
val labelColName = "fare_amount"

var featureNames = Array("h_distance", "pickup_longitude","pickup_latitude","dropoff_longitude", "dropoff_latitude","rate_code","hour", "day_of_week","trip_time")

val regressorParam = Map(
    "learning_rate" -> 0.05,
    "gamma" -> 1,
    "objective" ->"reg:gamma",
    "max_depth" -> 8,
    "subsample" -> 0.8,
    "num_round" -> 100,
    "tree_method" -> "gpu_hist")


val regressor = new XGBoostRegressor(regressorParam).setLabelCol(labelColName).setFeaturesCols(featureNames)

val paramGrid = new ParamGridBuilder().addGrid(regressor.maxDepth, Array(3, 8)).addGrid(regressor.eta, Array(0.2, 0.6)).build()

val evaluator = new RegressionEvaluator().setLabelCol(labelColName)
val cv = new CrossValidator().setEstimator(regressor).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3)

regressor.explainParams()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelColName: String = fare_amount
featureNames: Array[String] = Array(h_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, rate_code, hour, day_of_week, trip_time)
regressorParam: scala.collection.immutable.Map[String,Any] = Map(learning_rate -> 0.05, subsample -> 0.8, max_depth -> 8, objective -> reg:gamma, num_round -> 100, tree_method -> gpu_hist, gamma -> 1)
regressor: ml.dmlc.xgboost4j.scala.spark.XGBoostRegressor = xgbr_4be32fc7673f
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	xgbr_4be32fc7673f-eta: 0.2,
	xgbr_4be32fc7673f-maxDepth: 3
}, {
	xgbr_4be32fc7673f-eta: 0.6,
	xgbr_4be32fc7673f-maxDepth: 3
}, {
	xgbr_4be32fc7673f-eta: 0.2,
	xgbr_4be32fc7673f-maxDepth: 8
}, {
	xgbr_4be32fc7673f-eta: 0.6,
	xgbr_4be32fc7673f-maxDepth: 8
})
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_300a39d8d2b0, metricName=rmse, throughOrigin=false
cv: ml.dmlc.xgboost4j.scala.spark.rapids.CrossValidator 

In [18]:
val cvmodel = cv.fit(traindf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cvmodel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_0dd335ee52c8, bestModel=xgbr_4be32fc7673f, numFolds=3


In [19]:
val model = cvmodel.bestModel.asInstanceOf[XGBoostRegressionModel]
cvmodel.getEstimatorParamMaps.zip(cvmodel.avgMetrics)
model.extractParamMap()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model: ml.dmlc.xgboost4j.scala.spark.XGBoostRegressionModel = xgbr_4be32fc7673f
res25: Array[(org.apache.spark.ml.param.ParamMap, Double)] =
Array(({
	xgbr_4be32fc7673f-eta: 0.2,
	xgbr_4be32fc7673f-maxDepth: 3
},2.171293916330036), ({
	xgbr_4be32fc7673f-eta: 0.6,
	xgbr_4be32fc7673f-maxDepth: 3
},2.2360314763770623), ({
	xgbr_4be32fc7673f-eta: 0.2,
	xgbr_4be32fc7673f-maxDepth: 8
},2.111866299838413), ({
	xgbr_4be32fc7673f-eta: 0.6,
	xgbr_4be32fc7673f-maxDepth: 8
},2.3810327150036827))
res26: org.apache.spark.ml.param.ParamMap =
{
	xgbr_4be32fc7673f-alpha: 0.0,
	xgbr_4be32fc7673f-baseScore: 0.5,
	xgbr_4be32fc7673f-batchSize: 32768,
	xgbr_4be32fc7673f-buildAllColumnsInTransform: true,
	xgbr_4be32fc7673f-cacheTrainingSet: false,
	xgbr_4be32fc7673f-checkpointInterval: -1,
	xgbr_4be32fc7673f-checkpointPath: ,
	xgbr_4be32fc7673f-colsampleBylevel: 1.0,
	xgbr_4be32fc7673f-colsampleBytree: 1.0,
	xgbr_4be32fc7673f-customEval: null,
	xgbr_4be32fc7673f-customObj: null,
	xgbr_4be32fc7673f-eta: 0.2,


In [20]:
val pdf = model.transform(evaldf).cache()
pdf.select("fare_amount", "prediction").describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [pickup_longitude: float, pickup_latitude: float ... 10 more fields]
+-------+------------------+------------------+
|summary|       fare_amount|        prediction|
+-------+------------------+------------------+
|  count|              9016|              9016|
|   mean|  15.1896628216504| 15.14138584599203|
| stddev|10.615080872910664|10.419272301100078|
|    min|               2.5| 5.163010120391846|
|    max|              97.0|    80.55908203125|
+-------+------------------+------------------+



In [21]:
val evaluator = new RegressionEvaluator().setLabelCol(labelColName)
val rmse = evaluator.evaluate(pdf)
println(s"RMSE is $rmse")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_d8aba078c9b8, metricName=rmse, throughOrigin=false
rmse: Double = 2.0713523161267724
RMSE is 2.0713523161267724


In [22]:
val r2 = evaluator.setMetricName("r2").evaluate(pdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

r2: Double = 0.9619188981396372


In [26]:
var predictions = pdf.withColumn("error", $"prediction" - $"fare_amount")
val avgFare = tdf.select(avg("fare_amount")).first().getDouble(0)
predictions = predictions.withColumn("avgPrediction", lit(avgFare))
val regressionMeanEvaluator = new RegressionEvaluator().setPredictionCol("avgPrediction").setLabelCol("fare_amount").setMetricName("rmse")

val rmsep = regressionMeanEvaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [pickup_longitude: float, pickup_latitude: float ... 11 more fields]
avgFare: Double = 15.115734539162325
predictions: org.apache.spark.sql.DataFrame = [pickup_longitude: float, pickup_latitude: float ... 12 more fields]
regressionMeanEvaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_667cc91ea3de, metricName=rmse, throughOrigin=false
rmsep: Double = 10.61474962277036


In [27]:
val df=pdf
df.cache
df.createOrReplaceTempView("taxi")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [pickup_longitude: float, pickup_latitude: float ... 10 more fields]
res30: df.type = [pickup_longitude: float, pickup_latitude: float ... 10 more fields]


In [30]:
df.show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------+-----------------+----------------+-----------+---------+----+-----------+----+------------------+------------------+
|pickup_longitude|pickup_latitude|rate_code|dropoff_longitude|dropoff_latitude|fare_amount|trip_time| day|day_of_week|hour|        h_distance|        prediction|
+----------------+---------------+---------+-----------------+----------------+-----------+---------+----+-----------+----+------------------+------------------+
|      -74.018005|         40.705|      1.0|       -73.995995|          40.759|       14.5|      810|30.0|        2.0|14.0| 6.211866922231748|14.204110145568848|
|      -74.018005|         40.708|      1.0|          -73.955|       40.785004|       30.0|     1910|19.0|        5.0|19.0|10.043941644745642|32.801578521728516|
|      -74.017006|         40.705|      1.0|          -73.997|       40.725002|       12.0|      928|12.0|        5.0|14.0| 2.800500254046385|12.361300468444824|
|      -74.017006|         4

In [31]:
%%sql
select h_distance, avg(fare_amount) from taxi group by h_distance order by h_distance;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()